In [1]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline 
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
#有中文出现的情况，需要u'内容'

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from datum import *
import model
import imp

imp.reload(model)

<module 'model' from 'model.pyc'>

In [2]:
data = Datum()
data.data_prepare()
data.get_embedding('embedding_all.emb')
data.supervised_data_prepare()
data.evaluation_prepare()
data.label_trend()
data.label_return()

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import linear_model as LR
from sklearn import metrics as mt
import scipy.stats as stats

import collections
import math
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from six.moves import xrange

import datum

data_index = 0
len_stock = 3145
len_fund  = 2199

def stock2all(a):
    b = a.swapaxes(0, 2)
    c = b.reshape(b.shape[0], -1)
    d = c.T
    return d

a = np.array(range(27)).reshape((3, 3, 3))
b = stock2all(a)

class Model():
    def __init__(self, learning_rate_rank):
        self.save_rank = learning_rate_rank
        self.learning_rate = 1 / np.power(10, learning_rate_rank)
        self.batch_size = 10

    def data_initial(self, datum = None):
        if datum is not None:
            self.data = datum
        else:
            self.data = datum.Datum()
            self.data.data_prepare()
            self.data.evaluation_prepare()
            self.data.label_trend()
            self.data.label_return()
        
    def data_split(self):
        self.day_sample = self.data.price_data.shape[1]
        self.stock_sample = self.data.price_data.shape[0]
        
        self.train_vali = self.day_sample // 2
        self.vali_test = self.train_vali + self.day_sample // 4
        
        self.rank_day = np.array(range(self.train_vali))
        self.rank_stock = np.array(range(self.stock_sample))

        use_index = []
        for stock_idx in range(self.stock_sample):
            stock_name = str(self.data.code_tag[stock_idx])
            for _ in range(6-len(stock_name)):
                stock_name = '0' + stock_name
            if stock_name not in self.data.list_stocks:
                continue
            else:
                use_index.append(stock_idx)
        self.use_index = np.array(use_index)        
        
    def factor_network(self):
        learning_rate = self.learning_rate
        
        self.embedding = tf.placeholder(tf.float32, shape=[32, 1], name='embedding')
        self.factor = tf.placeholder(tf.float32, shape=[44, None], name='factor')
        self.factor_index = tf.placeholder(tf.int32, shape=[5], name='factor_index')
        self.label = tf.placeholder(tf.int32, shape=[None, 2], name='lable')
        
        self.u_bias = tf.get_variable('u_bias', shape=[44, 1, 32], initializer=tf.truncated_normal_initializer(stddev=1.0))
        self.weight = tf.get_variable(name='weight', shape=[32, 32], initializer=tf.truncated_normal_initializer(stddev=1.0))
        self.bias = tf.get_variable(name='bias', shape=[32], initializer=tf.zeros_initializer)
        self.project = tf.get_variable(name='project', shape=[32], initializer=tf.truncated_normal_initializer(stddev=1.0))
        
        self.u_bias_select = tf.nn.embedding_lookup(self.u_bias, self.factor_index)
        self.factor_select = tf.nn.embedding_lookup(self.factor, self.factor_index)
        
        concat = []
        
        for i in range(5):
            add1 = tf.transpose(tf.matmul(self.weight, self.embedding))
            add2 = tf.matmul(tf.expand_dims(self.factor_select[i, :], 1), self.u_bias_select[i])
            concat.append(tf.reduce_sum(self.project * tf.tanh(add1 + add2 + self.bias), axis=1, keep_dims=True))
        self.hidden = tf.concat([concat[i] for i in range(5)], axis=1)
        

        all_feature = tf.concat([tf.transpose(self.factor_select), self.hidden], 1)
        self.pred = tf.contrib.layers.fully_connected(
            inputs=all_feature,
            num_outputs=2,  # hidden
            activation_fn=tf.tanh,
            weights_initializer=tf.truncated_normal_initializer(stddev=1.0),
            biases_initializer=tf.zeros_initializer()
        )
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.pred, labels=self.label))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.cost)
        
        concat_all = []
        
        for i in range(44):
            add3 = tf.transpose(tf.matmul(self.weight, self.embedding))
            add4 = tf.matmul(tf.expand_dims(self.factor[i, :], 1), self.u_bias[i])
            concat_all.append(tf.reduce_sum(self.project * tf.tanh(add3 + add4 + self.bias), axis=1, keep_dims=True))
        self.evaluation = tf.concat([concat_all[i] for i in range(44)], axis=1)
        
    def training(self):
        epochs = 50
        batch_num = self.train_vali // self.batch_size
        saver = tf.train.Saver(max_to_keep=None)
        with tf.Session() as sess:
            saver.restore(sess, 'model_initial2/logmodel.ckpt')
            for epoch in range(epochs):
                print('epoch: {}'.format(epoch))
                # validation
                new_f_total = []
                for stock_idx in range(self.use_index):
                    price = self.data.price_data[stock_idx, self.train_vali:self.vali_test]
                    feature = self.data.feature_data[stock_idx, self.train_vali:self.vali_test, :]
                    label = self.data.ar_trend[stock_idx, self.train_vali:self.vali_test, :]
                    stock_name = str(self.data.code_tag[stock_idx])
                    embed = np.expand_dims(self.data.embedding[self.data.list_stocks.index(stock_name)], axis=1)
                    factor_index = random.sample(list(range(44)), 5)
                    feed_dict = {self.embedding: embed, self.factor: feature.T, self.factor_index: factor_index, self.label: label}
                    new_f= sess.run(self.evaluation, feed_dict=feed_dict)        
                    new_f_total.append(new_f)
                new_f_total = np.array(new_f_total)
                use_index= self.use_index
                ic = np.zeros((44, self.vali_test-self.train_vali, 4))
                for day in range(0, self.vali_test-self.train_vali):
                    for fac in range(44):
                        for id_idx in range(4):
                            rank_ic = stats.spearmanr(self.data.ar_ic[use_index, day, id_idx], new_f_total[:, day, fac])
                            ic[fac, day, id_idx] = rank_ic[0]
                ic = ic.mean(axis=1)
                print(ic)
                if not os.path.exists('model-2-'+str(self.save_rank)):
                    os.mkdir('model-2-'+str(self.save_rank))
                    os.mkdir('data/evaluation-2-'+str(self.save_rank))
                pd.DataFrame(ic).to_csv('data/evaluation-2-{}/epoch_evaluation_{}.csv'.format(self.save_rank, epoch))
                saver.save(sess, 'model-2-{}/logmodel.ckpt'.format(self.save_rank), global_step=epoch)
                
                np.random.shuffle(self.rank_day)
                loss_all = 0
                loss_count = 0
                for batch_count in range(batch_num):#batch_num
                    print('batch_count:{}'.format(batch_count))
                    np.random.shuffle(self.rank_stock)
                    for stock_count, stock_idx in enumerate(self.rank_stock):
                        price = self.data.price_data[stock_idx, self.rank_day[batch_count*self.batch_size:(batch_count+1)*self.batch_size]]
                        feature = self.data.feature_data[stock_idx, self.rank_day[batch_count*self.batch_size:(batch_count+1)*self.batch_size], :]
                        label = self.data.ar_trend[stock_idx, self.rank_day[batch_count*self.batch_size:(batch_count+1)*self.batch_size], :]
                        stock_name = str(self.data.code_tag[stock_idx])
                        for _ in range(6-len(stock_name)):
                            stock_name = '0' + stock_name
                        if stock_name not in self.data.list_stocks:
                            continue
                        else:
                            embed = np.expand_dims(self.data.embedding[self.data.list_stocks.index(stock_name)], axis=1)
                        for epoch_factor in range(30):
                            factor_index = random.sample(list(range(44)), 5)
                            feed_dict = {self.embedding: embed, self.factor: feature.T, self.factor_index: factor_index, self.label: label}
                            _, loss_val = sess.run([self.optimizer, self.cost], feed_dict=feed_dict)
                            loss_all += loss_val
                            loss_count += 1
                    print('avg_loss: {}'.format(loss_all/loss_count))
                    loss_all = 0
                    loss_count = 0
                
    def vali(self):
        epochs = 50
        batch_num = self.train_vali // self.batch_size
        saver = tf.train.Saver(max_to_keep=None)
        with tf.Session() as sess:
            for model_num in range(14, 15):
                saver.restore(sess, 'model-1-5/logmodel-{}.ckpt'.format(model_num))
                # validation
                new_f_total = []
                use_index = []
                for stock_idx in self.use_index:
                    price = self.data.price_data[stock_idx, :self.vali_test]
                    feature = self.data.feature_data[stock_idx, :self.vali_test, :]
                    label = self.data.ar_trend[stock_idx, :self.vali_test, :]
                    stock_name = str(self.data.code_tag[stock_idx])
                    for _ in range(6-len(stock_name)):
                        stock_name = '0' + stock_name                    
                    embed = np.expand_dims(self.data.embedding[self.data.list_stocks.index(stock_name)], axis=1)
                    factor_index = random.sample(list(range(44)), 5)
                    feed_dict = {self.embedding: embed, self.factor: feature.T, self.factor_index: factor_index, self.label: label}
                    new_f= sess.run(self.evaluation, feed_dict=feed_dict)        
                    new_f_total.append(new_f)
                new_f_total = np.array(new_f_total)
                use_index = self.use_index
                ic = np.zeros((44, self.vali_test-self.train_vali, 4))
                for day in range(self.train_vali, self.vali_test):
                    for fac in range(44):
                        for id_idx in range(4):
                            rank_ic = stats.spearmanr(self.data.ar_ic[use_index, day, id_idx], new_f_total[:, day, fac])
                            ic[fac, day-self.train_vali, id_idx] = rank_ic[0]
                ic = ic.mean(axis=1)
        ic = ic.mean(axis=1)
        self.minus_factor = np.where(ic < 0)[0] 

    def test(self):
        epochs = 50
        batch_num = self.train_vali // self.batch_size
        saver = tf.train.Saver(max_to_keep=None)
        with tf.Session() as sess:
            for model_num in range(20, 21):
                saver.restore(sess, 'model-1-4/logmodel-{}.ckpt'.format(model_num))
                # test
                new_f_total = []
                use_index = []
                for stock_idx in self.use_index:
                    price = self.data.price_data[stock_idx, :]
                    feature = self.data.feature_data[stock_idx, :, :]
                    label = self.data.ar_trend[stock_idx, :5, :]
                    stock_name = str(self.data.code_tag[stock_idx])
                    for _ in range(6-len(stock_name)):
                        stock_name = '0' + stock_name                       
                    embed = np.expand_dims(self.data.embedding[self.data.list_stocks.index(stock_name)], axis=1)
                    factor_index = random.sample(list(range(44)), 5)
                    feed_dict = {self.embedding: embed, self.factor: feature.T, self.factor_index: factor_index, self.label: label}
                    new_f= sess.run(self.evaluation, feed_dict=feed_dict)        
                    new_f_total.append(new_f)
                new_f_total = np.array(new_f_total)
                use_index = self.use_index
                ic = np.zeros((44, self.day_sample-self.vali_test, 4))
                for day in range(self.vali_test, self.day_sample):
                    for fac in range(44):
                        for id_idx in range(4):
                            rank_ic = stats.spearmanr(self.data.ar_ic[use_index, day, id_idx], new_f_total[:, day-self.vali_test, fac])
                            ic[fac, day-self.vali_test, id_idx] = rank_ic[0]
                ic = ic.mean(axis=1)
                print(ic)     
                print(np.abs(ic).mean())
        self.factor_test = new_f_total 
        self.ic = ic

In [4]:
tf.reset_default_graph()
model = Model(3)
model.data_initial(data)
model.data_split()
model.factor_network()
model.test()

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


INFO:tensorflow:Restoring parameters from model-1-4/logmodel-20.ckpt
[[-0.00736113 -0.01100544 -0.01673548 -0.02973511]
 [-0.00710425 -0.0109674  -0.01673646 -0.02940153]
 [-0.00650733 -0.0114733  -0.01714822 -0.02901271]
 [-0.00725789 -0.01106167 -0.01679869 -0.02962963]
 [-0.00715966 -0.01079498 -0.01660967 -0.0297128 ]
 [-0.00733805 -0.0110857  -0.01666332 -0.02939548]
 [-0.00718852 -0.01096334 -0.01698187 -0.03017095]
 [-0.0069088  -0.01059303 -0.01573201 -0.02771635]
 [-0.00767508 -0.01171664 -0.01715455 -0.029422  ]
 [-0.00733776 -0.01107937 -0.01666719 -0.02954589]
 [-0.00709149 -0.01111842 -0.01686594 -0.02956678]
 [-0.00750695 -0.01141749 -0.01713729 -0.02992967]
 [-0.00700912 -0.01066878 -0.01631177 -0.02898707]
 [-0.00748071 -0.01152072 -0.01739898 -0.03058096]
 [-0.00757238 -0.01157514 -0.01751313 -0.0312234 ]
 [-0.00806036 -0.01294658 -0.01939745 -0.03308506]
 [-0.01078128 -0.01516858 -0.01847731 -0.02899586]
 [-0.00559383 -0.00891496 -0.01348178 -0.02235469]
 [-0.00111774

In [6]:
def stock2all(a):
    b = a.swapaxes(0, 2)
    c = b.reshape(b.shape[0], -1)
    d = c.T
    return d

a = np.array(range(27)).reshape((3, 3, 3))
b = stock2all(a)


train_x = stock2all(data.feature_data[model.use_index, :model.train_vali, :])
train_y = np.argmax(stock2all(model.data.ar_trend[model.use_index, :model.train_vali, :]), axis=1)
test_x = stock2all(data.feature_data[model.use_index, model.vali_test:, :])
test_y = np.argmax(stock2all(model.data.ar_trend[model.use_index, model.vali_test:, :]), axis=1)

lr = LR.LogisticRegression()
cf_matrix = np.zeros((2, 2))
accuracy = []
f1_score = []

lr.fit(train_x, train_y)
predict_y = lr.predict(test_x)
cf_matrix += mt.confusion_matrix(test_y, predict_y)
accuracy.append(mt.accuracy_score(test_y, predict_y))
f1_score.append(mt.f1_score(test_y, predict_y))

print('accuracy', np.array(accuracy).mean())
print('f1_score', np.array(f1_score).mean())


train_x = np.concatenate([stock2all(model.factor_test[:, :model.train_vali, :]), train_x], axis=1)
test_x = np.concatenate([stock2all(model.factor_test[:, model.vali_test:, :]), test_x], axis=1)

lr = LR.LogisticRegression()
cf_matrix = np.zeros((2, 2))
accuracy = []
f1_score = []
lr.fit(train_x, train_y)
predict_y = lr.predict(test_x)
cf_matrix += mt.confusion_matrix(test_y, predict_y)
accuracy.append(mt.accuracy_score(test_y, predict_y))
f1_score.append(mt.f1_score(test_y, predict_y))
    
print('accuracy', np.array(accuracy).mean())
print('f1_score', np.array(f1_score).mean())



accuracy 0.548217744602
f1_score 0.576735179396
accuracy 0.548264987953
f1_score 0.577793232107
